# Lab 13 Part 3: Dashboard Routes and Customer APIs

**Duration:** 10 minutes  

## Learning Objectives

- Create main dashboard routes for different user roles
- Implement customer portal and data APIs
- Build customer graph visualization endpoints
- Design comprehensive customer overview services

## Import Required Dependencies

Ensure all necessary libraries and the connection manager are available.

In [ ]:
from fastapi import FastAPI, Request, WebSocket, WebSocketDisconnect
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
import json
import asyncio
from datetime import datetime, timedelta
from typing import List, Dict, Any
import uvicorn
from neo4j import GraphDatabase
import os

# Assumes app, templates, and connection_manager from previous notebooks
print("✓ Libraries imported successfully")

## Main Dashboard Routes

Define the primary routes for accessing different dashboards and user interfaces.

In [ ]:
@app.get("/", response_class=HTMLResponse)
async def dashboard_home(request: Request):
    """Main dashboard home page"""
    return templates.TemplateResponse("dashboard.html", {"request": request})

@app.get("/customer/{customer_id}", response_class=HTMLResponse)
async def customer_portal(request: Request, customer_id: str):
    """Customer portal interface"""
    return templates.TemplateResponse("customer_portal.html", {
        "request": request, 
        "customer_id": customer_id
    })

@app.get("/agent/{agent_id}", response_class=HTMLResponse)
async def agent_dashboard(request: Request, agent_id: str):
    """Agent dashboard interface"""
    return templates.TemplateResponse("agent_dashboard.html", {
        "request": request, 
        "agent_id": agent_id
    })

@app.get("/claims/{adjuster_id}", response_class=HTMLResponse)
async def claims_adjuster(request: Request, adjuster_id: str):
    """Claims adjuster interface"""
    return templates.TemplateResponse("claims_adjuster.html", {
        "request": request, 
        "adjuster_id": adjuster_id
    })

@app.get("/executive", response_class=HTMLResponse)
async def executive_dashboard(request: Request):
    """Executive dashboard interface"""
    return templates.TemplateResponse("executive_dashboard.html", {"request": request})

print("✓ Main dashboard routes configured")

## Customer Data APIs

Implement comprehensive customer data APIs for overview information and graph visualizations.

In [ ]:
@app.get("/api/customer/{customer_id}/overview")
async def get_customer_overview(customer_id: str):
    """Get comprehensive customer overview"""
    
    query = """
    MATCH (c:Customer {customer_id: $customer_id})
    OPTIONAL MATCH (c)-[:HOLDS_POLICY]->(p:Policy)
    OPTIONAL MATCH (c)-[:FILED_CLAIM]->(claim:Claim)
    OPTIONAL MATCH (c)-[:SERVICED_BY]->(agent:Agent)
    
    RETURN 
        c,
        collect(DISTINCT p) as policies,
        collect(DISTINCT claim) as claims,
        agent,
        count(DISTINCT p) as policy_count,
        count(DISTINCT claim) as claim_count,
        sum(p.annual_premium) as total_premium
    """
    
    with connection_manager.get_session() as session:
        result = session.run(query, {"customer_id": customer_id})
        record = result.single()
        
        if record:
            customer = dict(record["c"])
            policies = [dict(p) for p in record["policies"]]
            claims = [dict(c) for c in record["claims"]]
            agent = dict(record["agent"]) if record["agent"] else None
            
            return {
                "customer": customer,
                "policies": policies,
                "claims": claims,
                "agent": agent,
                "summary": {
                    "policy_count": record["policy_count"],
                    "claim_count": record["claim_count"],
                    "total_premium": record["total_premium"]
                }
            }
        else:
            return {"error": "Customer not found"}

@app.get("/api/customer/{customer_id}/graph")
async def get_customer_graph_data(customer_id: str):
    """Get customer network graph data for visualization"""
    
    query = """
    MATCH (c:Customer {customer_id: $customer_id})
    OPTIONAL MATCH (c)-[r1:HOLDS_POLICY]->(p:Policy)
    OPTIONAL MATCH (c)-[r2:FILED_CLAIM]->(claim:Claim)
    OPTIONAL MATCH (c)-[r3:SERVICED_BY]->(agent:Agent)
    OPTIONAL MATCH (agent)-[r4:WORKS_AT]->(branch:Branch)
    OPTIONAL MATCH (p)-[r5:COVERS]->(asset:Asset)
    
    RETURN c, r1, p, r2, claim, r3, agent, r4, branch, r5, asset
    """
    
    with connection_manager.get_session() as session:
        result = session.run(query, {"customer_id": customer_id})
        
        nodes = []
        edges = []
        node_ids = set()
        
        for record in result:
            # Process nodes
            for key in ["c", "p", "claim", "agent", "branch", "asset"]:
                if record[key] and record[key].element_id not in node_ids:
                    node = dict(record[key])
                    node["id"] = record[key].element_id
                    node["label"] = list(record[key].labels)[0]
                    nodes.append(node)
                    node_ids.add(record[key].element_id)
            
            # Process relationships
            for rel_key in ["r1", "r2", "r3", "r4", "r5"]:
                if record[rel_key]:
                    rel = record[rel_key]
                    edges.append({
                        "source": rel.start_node.element_id,
                        "target": rel.end_node.element_id,
                        "type": rel.type,
                        "properties": dict(rel)
                    })
        
        return {"nodes": nodes, "edges": edges}

print("✓ Customer data APIs implemented")

## Test Customer APIs

Test the customer overview and graph visualization endpoints with sample data.

In [ ]:
# Test customer overview API (simulated)
print("Testing customer data APIs...")
print("✓ Customer overview API endpoint: /api/customer/{customer_id}/overview")
print("✓ Customer graph API endpoint: /api/customer/{customer_id}/graph")
print("\nSample test URLs:")
print("  - http://localhost:8000/api/customer/CUST_001/overview")
print("  - http://localhost:8000/api/customer/CUST_001/graph")

## Summary

In this notebook, you:
- Created main dashboard routes for multiple user roles
- Implemented customer portal interface routes
- Built comprehensive customer overview API
- Created customer graph visualization API for D3.js
- Configured routes for agent, claims, and executive dashboards

Next: Proceed to notebook 04 to implement agent dashboard and claims adjuster tools.